Machine Learning Model Development 🏗️📊 Configuration

In [1]:
import pandas as pd  # Importa a biblioteca Pandas para manipulação de dados
import sqlite3  # Importa a biblioteca SQLite para interação com bancos de dados SQLite
import numpy as np  # Importa a biblioteca NumPy para operações numéricas
from sklearn.preprocessing import StandardScaler, QuantileTransformer  # Importa classes para escalonamento de dados
from datetime import datetime  # Importa a classe datetime para manipulação de datas e horários
from tqdm.auto import tqdm  # Importa a biblioteca tqdm para exibir barras de progresso
from sklearn.model_selection import train_test_split # Importa o módulo Scikit-learn para divisão de dados em conjuntos de treino e teste


DB_PATH = 'G:/Meu Drive/Documents/GitHubPublished/DataScienceProject/database/ecommerceProject.db'  # Define o caminho para o arquivo do banco de dados SQLite
conn = None  # Inicializa a variável de conexão com o banco de dados como None

def run_query(query: str, params=()):
    """
    Executa uma consulta SQL no banco de dados e retorna o resultado como um DataFrame.
    Adiciona tratamento de erro para a execução da query.

    Args:
        query (str): A consulta SQL a ser executada.
        params (tuple, optional): Parâmetros para a consulta SQL. Padrão é ().

    Returns:
        pandas.DataFrame: O resultado da consulta como um DataFrame, ou um DataFrame vazio em caso de erro.
    """
    global conn  # Permite que a função modifique a variável global conn
    if conn is None:  # Verifica se a conexão com o banco de dados ainda não foi estabelecida
        try:
            conn = sqlite3.connect(DB_PATH)  # Conecta ao banco de dados SQLite
        except sqlite3.Error as e:  # Captura erros relacionados à conexão com o banco de dados
            print(f"Erro ao conectar ao banco de dados: {e}")  # Imprime uma mensagem de erro
            return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

    try:
        df = pd.read_sql(query, conn, params=params)  # Executa a consulta SQL e armazena os resultados em um DataFrame
        return df  # Retorna o DataFrame com os resultados da consulta
    except Exception as e:  # Captura outros erros inesperados
        print(f"Erro ao executar a query: {query}\nErro: {e}")  # Imprime uma mensagem de erro
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

def load_required_data_with_progress():
    """
    Carrega todos os DataFrames necessários para a análise usando consultas SQL com barra de progresso.

    Returns:
        dict: Um dicionário onde as chaves são os nomes dos DataFrames e os valores são os DataFrames correspondentes.
    """
    queries = {
        'customers': 'SELECT * FROM customers',
        'orders': 'SELECT * FROM orders',
        'order_items': 'SELECT * FROM order_items',
        'products': 'SELECT * FROM products',
        'categories': 'SELECT * FROM categories'
    }  # Define um dicionário com as consultas SQL para carregar os dados
    data = {}  # Inicializa um dicionário para armazenar os DataFrames
    print("\n--- Carregando Dados Brutos ---")  # Imprime uma mensagem indicando que os dados brutos estão sendo carregados
    for key, query in tqdm(queries.items(), desc="Carregando tabelas do DB"):  # Itera sobre as consultas SQL com barra de progresso
        data[key] = run_query(query)  # Executa a consulta e armazena o resultado no dicionário
    return data  # Retorna o dicionário com os DataFrames

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Machine Learning Model Development 🏗️📊 Data Loading and Processing

In [2]:
# --- Carregar os dados brutos ---
raw_data = load_required_data_with_progress()  # Carrega os dados brutos do banco de dados
tCustomers = raw_data.get('customers', pd.DataFrame())  # Obtém o DataFrame de clientes do dicionário
tOrders = raw_data.get('orders', pd.DataFrame())  # Obtém o DataFrame de pedidos do dicionário
tOrderItens = raw_data.get('order_items', pd.DataFrame())  # Obtém o DataFrame de itens de pedido do dicionário
tProducts = raw_data.get('products', pd.DataFrame())  # Obtém o DataFrame de produtos do dicionário
tCategories = raw_data.get('categories', pd.DataFrame())  # Obtém o DataFrame de categorias do dicionário

if tCustomers.empty or tOrders.empty or tOrderItens.empty:  # Verifica se algum dos DataFrames essenciais não foi carregado
    print("Erro: Um ou mais DataFrames essenciais não foram carregados. Verifique o caminho do BD ou as queries.")  # Imprime uma mensagem de erro
    exit()  # Encerra o script

print("Dados brutos carregados com sucesso!")  # Imprime uma mensagem indicando que os dados brutos foram carregados com sucesso
print(f"Clientes: {len(tCustomers)} registros")  # Imprime o número de registros no DataFrame de clientes
print(f"Pedidos: {len(tOrders)} registros")  # Imprime o número de registros no DataFrame de pedidos
print(f"Itens de Pedido: {len(tOrderItens)} registros")  # Imprime o número de registros no DataFrame de itens de pedido

print("\n--- Limpeza e Tratamento de Dados ---")  # Imprime uma mensagem indicando que a limpeza e tratamento de dados estão sendo iniciados

tOrders['order_date'] = pd.to_datetime(tOrders['order_date'], errors='coerce')  # Converte a coluna 'order_date' para o tipo datetime

for df_name, df in tqdm({
    'tCustomers': tCustomers,
    'tOrders': tOrders,
    'tOrderItens': tOrderItens,
    'tProducts': tProducts,
    'tCategories': tCategories
}.items(), desc="Tratando valores ausentes"):  # Itera sobre os DataFrames com barra de progresso
    initial_rows = len(df)  # Armazena o número inicial de linhas no DataFrame
    for col in df.columns:  # Itera sobre as colunas do DataFrame
        if df[col].isnull().any():  # Verifica se a coluna possui valores ausentes
            if pd.api.types.is_numeric_dtype(df[col]):  # Verifica se a coluna é numérica
                df[col].fillna(df[col].median(), inplace=True)  # Preenche os valores ausentes com a mediana
            elif pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):  # Verifica se a coluna é do tipo string ou objeto
                df[col].fillna('Desconhecido', inplace=True)  # Preenche os valores ausentes com a string 'Desconhecido'
            else:  # Se a coluna não for numérica nem string
                df[col].fillna(df[col].mode()[0], inplace=True)  # Preenche os valores ausentes com a moda
    if len(df) == initial_rows:  # Verifica se o número de linhas no DataFrame não foi alterado
        pass  # Se o número de linhas não foi alterado, não faz nada
    else:  # Se o número de linhas foi alterado
        print(f"  {df_name}: {initial_rows - len(df)} linhas removidas devido a valores ausentes não tratáveis.")  # Imprime uma mensagem indicando que linhas foram removidas

for df_name, df, id_col in tqdm([
    ('tCustomers', tCustomers, 'customer_id'),
    ('tOrders', tOrders, 'order_id'),
    ('tOrderItens', tOrderItens, None),
    ('tProducts', tProducts, 'product_id'),
    ('tCategories', tCategories, 'category_id')
], desc="Tratando duplicatas"):  # Itera sobre os DataFrames com barra de progresso
    initial_rows = len(df)  # Armazena o número inicial de linhas no DataFrame
    if id_col and id_col in df.columns:  # Verifica se a coluna de ID foi especificada e existe no DataFrame
        df.drop_duplicates(subset=[id_col], inplace=True)  # Remove as linhas duplicadas com base na coluna de ID
        if len(df) < initial_rows:  # Verifica se o número de linhas no DataFrame foi alterado
            print(f"  {df_name}: {initial_rows - len(df)} duplicatas removidas baseadas em '{id_col}'.")  # Imprime uma mensagem indicando que linhas foram removidas
    else:  # Se a coluna de ID não foi especificada
        df.drop_duplicates(inplace=True)  # Remove as linhas duplicadas com base em todas as colunas
        if len(df) < initial_rows:  # Verifica se o número de linhas no DataFrame foi alterado
            print(f"  {df_name}: {initial_rows - len(df)} duplicatas de linha completa removidas.")  # Imprime uma mensagem indicando que linhas foram removidas
    if len(df) == initial_rows:  # Verifica se o número de linhas no DataFrame não foi alterado
        pass  # Se o número de linhas não foi alterado, não faz nada

print("\n--- Tratamento Específico de Outliers: 'order_value' ---")  # Imprime uma mensagem indicando que o tratamento de outliers está sendo iniciado
if not tOrders.empty and 'order_value' in tOrders.columns:  # Verifica se o DataFrame de pedidos não está vazio e se a coluna 'order_value' existe
    tOrders['order_value'] = pd.to_numeric(tOrders['order_value'], errors='coerce').fillna(tOrders['order_value'].median())  # Converte a coluna 'order_value' para o tipo numérico e preenche os valores ausentes com a mediana

    upper_bound_order_value = tOrders['order_value'].quantile(0.995)  # Calcula o limite superior para outliers
    lower_bound_order_value = tOrders['order_value'].quantile(0.005)  # Calcula o limite inferior para outliers

    initial_outliers = tOrders[(tOrders['order_value'] > upper_bound_order_value) | (tOrders['order_value'] < lower_bound_order_value)].shape[0]  # Calcula o número de outliers

    tOrders['order_value_cleaned'] = np.where(
        tOrders['order_value'] > upper_bound_order_value,
        upper_bound_order_value,
        np.where(
            tOrders['order_value'] < lower_bound_order_value,
            lower_bound_order_value,
            tOrders['order_value']
        )
    )  # Aplica a Winsorização para tratar os outliers
    print(f"  'order_value': {initial_outliers} outliers (acima de {upper_bound_order_value:.2f} ou abaixo de {lower_bound_order_value:.2f}) foram tratados via Winsorização.")  # Imprime uma mensagem indicando que os outliers foram tratados
else:  # Se o DataFrame de pedidos estiver vazio ou a coluna 'order_value' não existir
    print("  'order_value' não encontrado ou DataFrame de Pedidos vazio. Nenhum tratamento de outlier realizado.")  # Imprime uma mensagem indicando que nenhum tratamento de outlier foi realizado


--- Carregando Dados Brutos ---


Carregando tabelas do DB: 100%|██████████| 5/5 [01:22<00:00, 16.56s/it]


Dados brutos carregados com sucesso!
Clientes: 11161 registros
Pedidos: 274794 registros
Itens de Pedido: 825534 registros

--- Limpeza e Tratamento de Dados ---


Tratando duplicatas: 100%|██████████| 5/5 [00:07<00:00,  1.55s/it]



--- Tratamento Específico de Outliers: 'order_value' ---
  'order_value': 2748 outliers (acima de 19810.71 ou abaixo de 138.27) foram tratados via Winsorização.


Machine Learning Model Development 🏗️📊 Feature Engineering RFM

In [3]:
# --- 1.2. Engenharia de Atributos: Modelo RFM para Previsão de Churn ---
print("\n--- Engenharia de Atributos: Calculando RFM ---")  # Imprime uma mensagem indicando que a engenharia de atributos está sendo iniciada

if not tOrders.empty and 'order_date' in tOrders.columns and tOrders['order_date'].notna().any():  # Verifica se o DataFrame de pedidos não está vazio e se a coluna 'order_date' existe
    current_date = tOrders['order_date'].max() + pd.Timedelta(days=1)  # Calcula a data atual como a data máxima de pedido + 1 dia
else:  # Se o DataFrame de pedidos estiver vazio ou a coluna 'order_date' não existir
    print("  Aviso: 'order_date' não encontrado ou vazio. Usando data atual para Recência.")  # Imprime uma mensagem indicando que a data atual está sendo usada
    current_date = datetime.now()  # Define a data atual como a data e hora atuais

last_purchase = tOrders.groupby('customer_id')['order_date'].max().reset_index()  # Calcula a data da última compra para cada cliente
last_purchase.columns = ['customer_id', 'LastPurchaseDate']  # Define os nomes das colunas
tCustomers = pd.merge(tCustomers, last_purchase, on='customer_id', how='left')  # Mescla os DataFrames de clientes e últimas compras

if 'LastPurchaseDate' in tCustomers.columns:  # Verifica se a coluna 'LastPurchaseDate' existe no DataFrame de clientes
    tCustomers['Recency'] = (current_date - tCustomers['LastPurchaseDate']).dt.days  # Calcula a recência em dias
    max_recency_val = tCustomers['Recency'].max() if not tCustomers['Recency'].empty else 0  # Calcula o valor máximo de recência
    tCustomers['Recency'].fillna(max_recency_val + 30, inplace=True)  # Preenche os valores ausentes de recência com um valor alto
else:  # Se a coluna 'LastPurchaseDate' não existir no DataFrame de clientes
    tCustomers['Recency'] = max_recency_val + 30  # Define a recência como um valor alto

order_counts = tOrders.groupby('customer_id')['order_id'].count().reset_index()  # Calcula a frequência de compras para cada cliente
order_counts.columns = ['customer_id', 'Frequency']  # Define os nomes das colunas
tCustomers = pd.merge(tCustomers, order_counts, on='customer_id', how='left')  # Mescla os DataFrames de clientes e contagens de pedidos
tCustomers['Frequency'].fillna(0, inplace=True)  # Preenche os valores ausentes de frequência com 0

monetary_col = 'order_value_cleaned' if 'order_value_cleaned' in tOrders.columns else 'order_value'  # Define a coluna a ser usada para o cálculo do valor monetário
if monetary_col in tOrders.columns:  # Verifica se a coluna de valor monetário existe no DataFrame de pedidos
    total_monetary = tOrders.groupby('customer_id')[monetary_col].sum().reset_index()  # Calcula o valor total gasto por cada cliente
    total_monetary.columns = ['customer_id', 'Monetary']  # Define os nomes das colunas
    tCustomers = pd.merge(tCustomers, total_monetary, on='customer_id', how='left')  # Mescla os DataFrames de clientes e valores monetários
    tCustomers['Monetary'].fillna(0, inplace=True)  # Preenche os valores ausentes de valor monetário com 0
else:  # Se a coluna de valor monetário não existir no DataFrame de pedidos
    print(f"  Aviso: Coluna '{monetary_col}' não encontrada para cálculo de Valor Monetário. 'Monetary' definido como 0 para todos.")  # Imprime uma mensagem indicando que a coluna não foi encontrada
    tCustomers['Monetary'] = 0  # Define o valor monetário como 0 para todos os clientes

print("\nPrimeiras linhas do DataFrame de Clientes com atributos RFM:")  # Imprime um cabeçalho
print(tCustomers[['customer_id', 'Recency', 'Frequency', 'Monetary']].head())  # Imprime as primeiras linhas do DataFrame com os atributos RFM

features_for_churn = ['Recency', 'Frequency', 'Monetary']  # Define as colunas a serem usadas para modelagem de churn
existing_features = [f for f in features_for_churn if f in tCustomers.columns]  # Filtra as colunas que realmente existem no DataFrame

if not existing_features:  # Verifica se nenhuma das colunas RFM existe no DataFrame
    print("\nErro: Nenhum dos atributos RFM esperados foi encontrado. Verifique a engenharia de atributos.")  # Imprime uma mensagem de erro
    exit()  # Encerra o script

X = tCustomers[existing_features].copy()  # Cria uma cópia do DataFrame com as colunas RFM

print(f"\nAtributos selecionados para a modelagem: {existing_features}")  # Imprime as colunas selecionadas para a modelagem

print("\n--- Escalonando Atributos Numéricos ---")  # Imprime uma mensagem indicando que o escalonamento dos atributos está sendo iniciado

scaler = QuantileTransformer(output_distribution='normal', random_state=42)  # Cria uma instância do QuantileTransformer

if not X.empty:  # Verifica se o DataFrame com os atributos RFM não está vazio
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)  # Escala os atributos RFM usando o QuantileTransformer
    print("\nPrimeiras linhas dos atributos RFM escalados:")  # Imprime um cabeçalho
    print(X_scaled.head())  # Imprime as primeiras linhas dos atributos RFM escalados
else:  # Se o DataFrame com os atributos RFM estiver vazio
    print("  DataFrame de atributos X está vazio. Escalonamento não realizado.")  # Imprime uma mensagem indicando que o escalonamento não foi realizado
    X_scaled = pd.DataFrame(columns=existing_features)  # Cria um DataFrame vazio com as colunas RFM

base_cols = ['customer_id', 'customer_name']  # Define as colunas base a serem mantidas
cols_to_merge = [col for col in base_cols if col in tCustomers.columns]  # Filtra as colunas base que realmente existem no DataFrame
tCustomers_processed = tCustomers[cols_to_merge].copy()  # Cria uma cópia do DataFrame com as colunas base

for col in existing_features:  # Itera sobre as colunas RFM
    tCustomers_processed[col] = X[col]  # Adiciona a coluna RFM ao DataFrame processado
    if col in X_scaled.columns:  # Verifica se a coluna RFM escalada existe no DataFrame escalado
        tCustomers_processed[f'{col}_scaled'] = X_scaled[col]  # Adiciona a coluna RFM escalada ao DataFrame processado

print("\nDataFrame final de clientes processados com RFM e RFM escalados:")  # Imprime um cabeçalho
print(tCustomers_processed.head())  # Imprime as primeiras linhas do DataFrame processado


--- Engenharia de Atributos: Calculando RFM ---


C:\Users\santo\AppData\Local\Temp\ipykernel_4928\4166753065.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tCustomers['Recency'].fillna(max_recency_val + 30, inplace=True)  # Preenche os valores ausentes de recência com um valor alto
C:\Users\santo\AppData\Local\Temp\ipykernel_4928\4166753065.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the interme


Primeiras linhas do DataFrame de Clientes com atributos RFM:
                            customer_id  Recency  Frequency   Monetary
0  80ab0365-8249-48b9-aa62-d3b71868b7a2       21         24  134970.35
1  60940726-e1e6-4528-972d-4358ed736124        6         19  137214.48
2  c083eb12-9c91-4fcc-8a82-da28f2d8f01f       20         26  138579.58
3  21d02d02-2d3d-484f-8d23-a66840eac57b       22         20  125973.50
4  0aa2e281-4934-4303-bb70-0e2b1cbf25bf       43         32  208464.15

Atributos selecionados para a modelagem: ['Recency', 'Frequency', 'Monetary']

--- Escalonando Atributos Numéricos ---

Primeiras linhas dos atributos RFM escalados:
    Recency  Frequency  Monetary
0 -0.423855  -0.085414 -0.667908
1 -1.258508  -1.140077 -0.601248
2 -0.461215   0.304482 -0.560961
3 -0.387078  -0.926176 -0.922123
4  0.180377   1.435477  1.185430

DataFrame final de clientes processados com RFM e RFM escalados:
                            customer_id       customer_name  Recency  \
0  80ab03

Machine Learning Model Development 🏗️📊 Churn Target Definition

In [4]:
print("\n--- Definindo a Variável Alvo 'Churn' ---")  # Imprime uma mensagem indicando que a definição da variável alvo está sendo iniciada

CHURN_THRESHOLD_DAYS = 90  # Define o limiar de recência em dias para considerar um cliente como churn

tCustomers_processed['is_churn'] = (tCustomers_processed['Recency'] > CHURN_THRESHOLD_DAYS).astype(int)  # Cria a coluna 'is_churn' indicando se o cliente é churn ou não

print(f"Clientes com Recency > {CHURN_THRESHOLD_DAYS} dias são considerados Churn (is_churn = 1).")  # Imprime uma mensagem indicando o critério para definir churn
print("\nDistribuição da variável 'is_churn':")  # Imprime um cabeçalho
print(tCustomers_processed['is_churn'].value_counts())  # Imprime a distribuição da variável 'is_churn'
print(f"Proporção de Churn: {tCustomers_processed['is_churn'].mean():.2f}")  # Imprime a proporção de clientes churn


--- Definindo a Variável Alvo 'Churn' ---
Clientes com Recency > 90 dias são considerados Churn (is_churn = 1).

Distribuição da variável 'is_churn':
is_churn
0    9301
1    1860
Name: count, dtype: int64
Proporção de Churn: 0.17


Machine Learning Model Development 🏗️📊 Train Test Split

In [5]:
print("\n--- Dividindo os Dados em Conjuntos de Treino e Teste ---")  # Imprime uma mensagem indicando que a divisão dos dados está sendo iniciada

X = tCustomers_processed[[f'{col}_scaled' for col in existing_features]].copy()  # Usar as features escaladas
y = tCustomers_processed['is_churn'].copy()  # Copia a variável alvo

from sklearn.model_selection import train_test_split  # Importa a função train_test_split para dividir os dados

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)  # Divide os dados em conjuntos de treino e teste

print(f"Tamanho total do dataset: {len(X)} clientes")  # Imprime o tamanho total do dataset
print(f"Tamanho do conjunto de Treino (X_train, y_train): {len(X_train)} clientes")  # Imprime o tamanho do conjunto de treino
print(f"Tamanho do conjunto de Teste (X_test, y_test): {len(X_test)} clientes")  # Imprime o tamanho do conjunto de teste

print("\nProporção de Churn no conjunto de Treino:")  # Imprime um cabeçalho
print(y_train.value_counts(normalize=True))  # Imprime a proporção de churn no conjunto de treino

print("\nProporção de Churn no conjunto de Teste:")  # Imprime um cabeçalho
print(y_test.value_counts(normalize=True))  # Imprime a proporção de churn no conjunto de teste


--- Dividindo os Dados em Conjuntos de Treino e Teste ---
Tamanho total do dataset: 11161 clientes
Tamanho do conjunto de Treino (X_train, y_train): 8928 clientes
Tamanho do conjunto de Teste (X_test, y_test): 2233 clientes

Proporção de Churn no conjunto de Treino:
is_churn
0    0.833333
1    0.166667
Name: proportion, dtype: float64

Proporção de Churn no conjunto de Teste:
is_churn
0    0.833408
1    0.166592
Name: proportion, dtype: float64


Machine Learning Model Development 🏗️📊 Churn Prediction Model

In [6]:
if conn:  # Verifica se a conexão com o banco de dados foi estabelecida
    conn.close()  # Fecha a conexão com o banco de dados
    print("\nConexão com o banco de dados SQLite fechada.")  # Imprime uma mensagem indicando que a conexão foi fechada


Conexão com o banco de dados SQLite fechada.
